In this document a brief usage manual for the TSFEDL library under the S-ADL library.

First we are going to load some data from the KDDCup99 dataset, which is currently used for testing and is also inside the library as an utility.

## Downloading the dataset KDD99Cup

First, we import the dataset utility for time series data

In [18]:
import SADL.time_series.time_series_datasets as dataset

Dataset loading process

In [1]:
import os
import sklearn 
import numpy as np
import pickle as pkl

batch_size = 32

if not os.path.exists(os.path.join(os.getcwd(),'test.pkl')):
    data, attack_types, classes = dataset.readKDDCup99Dataset()
    data = sklearn.preprocessing.StandardScaler().fit_transform(data)

    attack_types_dict = {}
    cont=0
    for att in attack_types:
        attack_types_dict[att]=cont
        cont+=1

    for i in range(len(classes)):
        classes[i] = attack_types_dict[classes[i]]

    classes[classes!=attack_types_dict["normal"]] = 1
    classes[classes==attack_types_dict["normal"]] = 0
    classes_test = classes[-500000:]

    normal_train = np.where(classes[:-500000]==0)[0][:10000]
    train_data = data[normal_train]

    with open('test.pkl','wb') as f:
        pkl.dump(train_data, f)
else: 
    with open('test.pkl','rb') as f:
        train_data = pkl.load(f)

## S-ADL/TSFEDL implemented models

S-ADL has coverage for 23 models implemented in TSFEDL Library. You can instantiate these models by adding the parameter model="" to the kwargs object. The dictionary of the available models is: 

    "ohshulih" : OhShuLih,
    "yibogao": YiboGao,
    "liohshu": LihOhShu,
    "yaoqihang" : YaoQihang,
    "htetmyetlynn" : HtetMyetLynn,
    "yildirimozal" : YildirimOzal,
    "caiwenjuan" : CaiWenjuan,
    "zhangjin" : ZhangJin,
    "kongzhengmin": KongZhengmin,
    "weixiaoyan" : WeiXiaoyan,
    "gaojunli": GaoJunLi,
    "khanzulfiqar" : KhanZulfiqar,
    "zhengzhenyu": ZhengZhenyu,
    "wangkejun" : WangKejun,
    "chenchen": ChenChen,
    "kimtaeyoung" : KimTaeYoung,
    "genminxing": GenMinxing,
    "fujiangmeng" : FuJiangmeng,
    "shihaotian" : ShiHaotian, 
    "huangmeiling": HuangMeiLing,
    "hongtan": HongTan,
    "sharpar" : SharPar,
    "daixili": DaiXiLi

## Testing a model: OhShuLih

We will use TimeSeriesDatasetV2 custom Dataset to create an appropiate tensor for time series tasks.

We will use TSFEDL_TopModule custom top_module proposed as a default nn.Module class to reshape the data into (batch_size, npred, out_features) to perform anomaly detection tasks on time series data.

We will use tsfedl library implemented via S-ADL

In [2]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
from SADL.time_series.time_series_utils import TSFEDL_TopModule
import torch
from SADL.time_series.algorithms import tsfedl

train_dataset = TimeSeriesDatasetV2(torch.from_numpy(train_data).double(), window_size=4, forecast_size=1, permute_size = (1,0))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle = True)

kwargs = {"algorithm_": "ohshulih", "loss": torch.nn.MSELoss(),"top_module": TSFEDL_TopModule(in_features=20, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model1 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model1.model = model1.model.double()

model1.fit(train_loader)

2024-03-05 11:02:06.982794: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 11:02:06.982828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 11:02:06.984022: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 11:02:06.991589: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-05 11:02:08.216575: W tensorflow/compiler/tf2

{'label_parser': True, 'algorithm_': 'OhShuLih', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}


/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You d

Training: |                                                                                           | 0/? [0…

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([12, 1, 126])) that is different to the input size (torch.Size([12, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.
